In [1]:
import pandas as pd
import numpy as np
import ast
import gzip
import json
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

In [8]:
user_item = pd.read_csv('../Datasets/user_items_cleaned.csv.gz')
steam_games = pd.read_csv ('../Datasets/steam_games_cleaned.csv')
user_reviews = pd.read_csv ('../Datasets/user_review_cleaned.csv')

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [10]:
def PlayTimeGenre(genero: str):
    # Merge entre user_items y steam_games
    merged_df = pd.merge(user_item, steam_games, left_on='item_name', right_on='app_name', how='inner')

    # Filtrar por el género específico
    genre_df = merged_df[merged_df['genres'].apply(lambda x: genero in x)]

    # Agrupar por año y sumar las horas jugadas
    grouped_df = genre_df.groupby('year')['playtime_forever'].sum()

    # Encontrar el año con más horas jugadas
    max_playtime_year = grouped_df.idxmax()

    # Retornar el resultado
    result = {"Año de lanzamiento con más horas jugadas para Género X": max_playtime_year}
    return result


In [11]:
genre_result = PlayTimeGenre("Action")
genre_result

{'Año de lanzamiento con más horas jugadas para Género X': 2012}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}



In [168]:
merged_df = pd.merge(user_item, steam_games, left_on='item_id', right_on='id')

In [13]:
def UserForGenre(genero):
    merge = merged_df
    
    usuario_agrupado = merge[['user_id','playtime_forever','year']].groupby(['user_id','year']).sum()
    usuario_agrupado_1 = usuario_agrupado['playtime_forever'].idxmax()
    
    genero = merge[merge['genres'].apply(lambda x: genero in x)]
    
    df_usuario_genero = merged_df[(merged_df['user_id'] == usuario_agrupado_1 [0])]
    poranio = df_usuario_genero.groupby('year')[['playtime_forever']].sum().reset_index().rename(columns={'year':'Año', 'playtime_forever': 'Horas jugadas'}).to_dict(orient='records')
    
    # Crear un diccionario con la información
    dicc = {
        'Uruario con mas horas jugadas por genero': usuario_agrupado_1 [0],
        'Horas Jugadas': poranio
    }

    return dicc
UserForGenre('Action')

{'Uruario con mas horas jugadas por genero': 'REBAS_AS_F-T',
 'Horas Jugadas': [{'Año': 1996, 'Horas jugadas': 0.0},
  {'Año': 1998, 'Horas jugadas': 1.0},
  {'Año': 1999, 'Horas jugadas': 225.0},
  {'Año': 2000, 'Horas jugadas': 0.0},
  {'Año': 2001, 'Horas jugadas': 11.0},
  {'Año': 2002, 'Horas jugadas': 2.0},
  {'Año': 2003, 'Horas jugadas': 1863.0},
  {'Año': 2004, 'Horas jugadas': 9102.0},
  {'Año': 2005, 'Horas jugadas': 3294.0},
  {'Año': 2006, 'Horas jugadas': 1811.0},
  {'Año': 2007, 'Horas jugadas': 1972.0},
  {'Año': 2008, 'Horas jugadas': 3814.0},
  {'Año': 2009, 'Horas jugadas': 34359.0},
  {'Año': 2010, 'Horas jugadas': 55012.0},
  {'Año': 2011, 'Horas jugadas': 151401.0},
  {'Año': 2012, 'Horas jugadas': 182758.0},
  {'Año': 2013, 'Horas jugadas': 330971.0},
  {'Año': 2014, 'Horas jugadas': 383089.0},
  {'Año': 2015, 'Horas jugadas': 965353.0},
  {'Año': 2016, 'Horas jugadas': 1169773.0},
  {'Año': 2017, 'Horas jugadas': 42390.0}]}

In [204]:
UserForGenre('Action')

{'Uruario con mas horas jugadas por genero': 'REBAS_AS_F-T',
 'Horas Jugadas': [{'Año': 1996, 'Horas jugadas': 0.0},
  {'Año': 1998, 'Horas jugadas': 0.0166666666666666},
  {'Año': 1999, 'Horas jugadas': 3.7499999999999996},
  {'Año': 2000, 'Horas jugadas': 0.0},
  {'Año': 2001, 'Horas jugadas': 0.1833333333333333},
  {'Año': 2002, 'Horas jugadas': 0.0166666666666666},
  {'Año': 2003, 'Horas jugadas': 31.05},
  {'Año': 2004, 'Horas jugadas': 151.70000000000002},
  {'Año': 2005, 'Horas jugadas': 54.88333333333333},
  {'Año': 2006, 'Horas jugadas': 30.78333333333333},
  {'Año': 2007, 'Horas jugadas': 173.76666666666665},
  {'Año': 2008, 'Horas jugadas': 83.88333333333333},
  {'Año': 2009, 'Horas jugadas': 1028.7},
  {'Año': 2010, 'Horas jugadas': 1215.9666666666667},
  {'Año': 2011, 'Horas jugadas': 3101.3666666666663},
  {'Año': 2012, 'Horas jugadas': 4459.333333333333},
  {'Año': 2013, 'Horas jugadas': 6723.35},
  {'Año': 2014, 'Horas jugadas': 7967.266666666666},
  {'Año': 2015, 'Hora

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]



In [14]:
def UsersRecommend(año: int):
    # 1. Realizar un pd.merge entre steam_games y user_review
    merged_df = pd.merge(steam_games, user_reviews, left_on='id', right_on='item_id', how='inner')
   
    # 2. Filtrar para incluir solo recomendaciones positivas/neutrales
    filtered_df = merged_df[(merged_df['recommend'] == 1) & (merged_df['sentiment_analysis'].isin([1, 2])) & (merged_df['year'] == año)]

    # 3. Agrupar por el nombre del juego, contar la cantidad de recomendaciones y ordenar en orden descendente
    game_recommendations = filtered_df.groupby('app_name')['recommend'].count().sort_values(ascending=False)

    # 4. Seleccionar los tres juegos principales
    top_3_games = game_recommendations.head(3).index.tolist()

    # Crear el formato de retorno
    result = [{"Puesto {}: {}".format(i + 1, juego)} for i, juego in enumerate(top_3_games)]
    
    return result


In [31]:
def UsersRecommend(year: int):
    try:   
        function3 = pd.read_csv('../Datasets/function3.csv')
    
        # 2. Filtrar para incluir solo recomendaciones positivas/neutrales
        filtered_df = function3[(function3['recommend'] == 1) & (function3['sentiment_analysis'].isin([1, 2])) & (function3['year'] == año)]

        # 3. Agrupar por el nombre del juego, contar la cantidad de recomendaciones y ordenar en orden descendente
        game_recommendations = filtered_df.groupby('app_name')['recommend'].count().sort_values(ascending=False)

        # 4. Seleccionar los tres juegos principales
        top_3_games = game_recommendations.head(3).index.tolist()

        # Crear el formato de retorno
        result = [{"Puesto {}: {}".format(i + 1, juego)} for i, juego in enumerate(top_3_games)]
        
        return result

    except Exception as e:
        return {"Error": str(e)}
    
type(UsersRecommend(2014))

list

In [15]:
# Ejemplo de uso
recommendations_result = UsersRecommend(2014)
print(recommendations_result)

[{'Puesto 1: Loadout'}, {'Puesto 2: Goat Simulator'}, {'Puesto 3: Insurgency'}]


def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]



In [35]:
def UsersNotRecommend(año: int):
    # 1. Realizar un pd.merge entre steam_games y user_review
    merged_df = pd.merge(steam_games, user_reviews, left_on='id', right_on='item_id', how='inner')
   
    # 2. Filtrar para incluir solo no recomendaciones y comentarios negativos para el año dado
    filtered_df = merged_df[(merged_df['recommend'] == 0) & (merged_df['sentiment_analysis'] == 0) & (merged_df['year'] == año)]

    # 3. Agrupar por el nombre del juego, contar la cantidad de no recomendaciones y ordenar en orden descendente
    game_not_recommendations = filtered_df.groupby('app_name')['recommend'].count().sort_values(ascending=False)

    # 4. Seleccionar los tres juegos principales, manejar empates
    top_3_not_recommendations = game_not_recommendations.head(3).index.tolist()

    # Crear el formato de retorno
    result = [{"Puesto {}: {}".format(i + 1, juego)} for i, juego in enumerate(top_3_not_recommendations)]
    
    return result

# Ejemplo de uso
not_recommendations_result = UsersNotRecommend(2014)
print(not_recommendations_result)


[{'Puesto 1: Call of Duty®: Ghosts'}, {'Puesto 2: Loadout'}, {'Puesto 3: theHunter Classic'}]


def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [38]:
def sentiment_analysis(año: int):
    # 1. Realizar un pd.merge entre steam_games y user_review
    merged_df = pd.merge(steam_games, user_reviews, left_on='id', right_on='item_id', how='inner')
   
    # 2. Filtrar para incluir solo las reseñas para el año dado
    filtered_df = merged_df[merged_df['year'] == año]

    # 3. Contar la cantidad de registros según el análisis de sentimiento
    sentiment_counts = filtered_df['sentiment_analysis'].value_counts()

    # 4. Crear el formato de retorno
    result = {
        'Negative': sentiment_counts.get(0, 0),
        'Neutral': sentiment_counts.get(1, 0),
        'Positive': sentiment_counts.get(2, 0)
    }
    
    return result

# Ejemplo de uso
sentiment_result = sentiment_analysis(2010)
print(sentiment_result)


{'Negative': 216, 'Neutral': 717, 'Positive': 852}


Si es un sistema de recomendación item-item:

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.


Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.